In [ ]:
!pip install uvicorn
!pip install fastapi
!pip install nest-asyncio pyngrok

### 라이브러리 선언

In [ ]:
import pandas as pd
import numpy as np
# 서버 관리용 fastapi 의존 라이브러리
import uvicorn
# 비동기형 통신 파이썬 라이브러리
from fastapi import FastAPI
# API 연계 데이터 정의
from pydantic import BaseModel

In [ ]:
from fastapi.middleware.cors import CORSMiddleware
origins = ["*"]

app = FastAPI(title="ML API")

# CORS 미들웨어 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 origin 허용
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

In [ ]:
import pickle

### 1. 모델 불러오기

In [ ]:
with open ("coreDtModel.pickle","rb") as fr:
    loadedModelData = pickle.load(fr)
loadedModelData
loadedModel = loadedModelData["model"]
features = loadedModelData["feature"]
label = loadedModelData["label"]
preprocessing = loadedModelData["preprocessing"]
inHclus = 1
inPropercent = 0.4
inPromotion = "Y"
inHoliday = "N"
ynDict = preprocessing[2]
inPromotion = ynDict[ inPromotion ]
inHoliday = ynDict[ inHoliday ]
testDf = pd.DataFrame([[ inHclus, inPropercent, inPromotion, inHoliday]])
predictValue = loadedModel.predict(testDf)[0]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [ ]:
predictValue

1955.0

### 2. 인터페이스 데이터 정의

In [ ]:
class InDataset(BaseModel):
    inHclus : int
    inPropercent : float
    inPromotion : str
    inHoliday : str

### 3. 예측용 함수 정의

In [ ]:
@app.post("/predict", status_code=200)
async def predict_ml( x: InDataset):
    inHclus = x.inHclus
    inPropercent = x.inPropercent
    inPromotion = x.inPromotion
    inHoliday = x.inHoliday
    # inHclus = 1
    # inPropercent = 0.4
    # inPromotion = "Y"
    # inHoliday = "N"
    ynDict = preprocessing[2]
    inPromotion = ynDict[ inPromotion ]
    inHoliday = ynDict[ inHoliday ]
    testDf = pd.DataFrame([[ inHclus, inPropercent, inPromotion, inHoliday]])
    predictValue = loadedModel.predict(testDf)[0]
    return {"prediction": predictValue}

In [ ]:
@app.get("/")
async def root():
    return {"message":"server is running"}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
auth_token = "본인 API 키"
ngrok.set_auth_token(auth_token)
ngrokTunnel = ngrok.connect(9999)
print("공용 URL", ngrokTunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=9999)

INFO:     Started server process [679]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:9999 (Press CTRL+C to quit)


공용 URL https://df2b-34-168-48-83.ngrok-free.app
INFO:     59.9.202.79:0 - "GET / HTTP/1.1" 200 OK
INFO:     59.9.202.79:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     59.9.202.79:0 - "POST /predict HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


INFO:     59.9.202.79:0 - "POST /predict HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [679]
